In [6]:
import ML_for_WDN
import numpy as np
import pandas as pd
import os

# Set working directory
abs_path = '/home/ntm/PhD/ML-for-WDN/notebooks/'
os.chdir(abs_path + '..')


In [10]:
# Load data
df = pd.read_csv(
    'data/raw_data/Leak_Location_1.csv', 
    skiprows=1
)